In [1]:
!pip install -U datasets trl accelerate peft bitsandbytes packaging ninja sentencepiece transformers einops trl huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.1/244.1 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 13.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.5/53.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.0/102.0 kB 7.1 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.28.0
    Uninstalling accelerate-0.28.0:
      Successfully uninstalled accelerate-0.28.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are in

In [2]:
!pip install tqdm scipy

In [3]:
import os
from dataclasses import dataclass, field
from typing import Optional

import torch
from datasets import load_dataset
from datasets import load_from_disk
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
import safetensors
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    AutoTokenizer,
    TrainingArguments,
)
from tqdm.notebook import tqdm

from trl import SFTTrainer

2024-04-12 11:54:47.468144: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-12 11:54:47.468271: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-12 11:54:47.595974: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
from huggingface_hub import interpreter_login
interpreter_login()



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .


Enter your token (input will not be visible):  ·····································
Add token as git credential? (Y/n)  y


Token is valid (permission: read).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.
Token has not been saved to git credential helper.
Your token has been saved to /root/.cache/huggingface/token
Login successful


## Load Dataset

In [ ]:
dataset = load_dataset("nbertagnolli/counsel-chat", split="train")
dataset

In [6]:
import pandas as pd
# Convert to DataFrame
df = pd.DataFrame(dataset)

# Display the first few rows of the DataFrame
df.head(2)

,questionID,questionTitle,questionText,questionLink,topic,therapistInfo,therapistURL,answerText,upvotes,views
0,0,Do I have too many issues for counseling?,I have so many issues to address. I have a his...,https://counselchat.com/questions/do-i-have-to...,depression,Jennifer MolinariHypnotherapist & Licensed Cou...,https://counselchat.com/therapists/jennifer-mo...,It is very common for people to have multiple ...,3,1971
1,0,Do I have too many issues for counseling?,I have so many issues to address. I have a his...,https://counselchat.com/questions/do-i-have-to...,depression,"Jason Lynch, MS, LMHC, LCAC, ADSIndividual & C...",https://counselchat.com/therapists/jason-lynch...,"I've never heard of someone having ""too many i...",2,386


## Create Train and Test Dataset

In [7]:

# Filter the required columns
#filtered_df = df[['questionText', 'topic']].drop_duplicates()
filtered_df = df[['questionText', 'topic', 'answerText']].drop_duplicates(subset=['questionText', 'topic'])
# Rename the columns
filtered_df.columns = ['Context', 'topic', 'Response']

# Group by topic and count the occurrences
topic_counts = filtered_df['topic'].value_counts()

# Calculate the target number of samples per topic for the test set
target_test_size_per_topic = (topic_counts * 0.2).round().astype(int)

# Initialize an empty DataFrame for the test set
test_set_balanced = pd.DataFrame(columns=filtered_df.columns)

# Initialize an empty DataFrame for the train set
train_set_balanced = pd.DataFrame(columns=filtered_df.columns)

# For each topic, randomly select the calculated number of samples to include in the test set
for topic, target_size in target_test_size_per_topic.items():
    samples = filtered_df[filtered_df['topic'] == topic].sample(n=min(target_size, topic_counts[topic]), random_state=42)
    test_set_balanced = pd.concat([test_set_balanced, samples])

        # Add the remaining samples to the train set
    train_set_balanced = pd.concat([train_set_balanced, df[df['topic'] == topic].drop(samples.index)])

print("train data shape",train_set_balanced.shape )
print("test data shape",test_set_balanced.shape )



train data shape (2599, 12)
test data shape (176, 3)


In [8]:

# Save the balanced test set
test_set_balanced.to_csv('counsel_chat_test_balanced.csv', index=False)

# Save the balanced train set
train_set_balanced.to_csv('counsel_chat_train_balanced.csv', index=False)


# Check the final distribution of topics in the balanced test set
balanced_test_distribution = test_set_balanced['topic'].value_counts()

print(balanced_test_distribution)
print("test data shape: \n",test_set_balanced.head() )

topic
depression                  28
intimacy                    22
relationships               21
anxiety                     20
family-conflict             12
parenting                   11
self-esteem                  9
relationship-dissolution     7
behavioral-change            6
anger-management             5
trauma                       5
marriage                     4
lgbtq                        3
domestic-violence            3
social-relationships         3
workplace-relationships      2
substance-abuse              2
grief-and-loss               2
spirituality                 2
counseling-fundamentals      1
professional-ethics          1
legal-regulatory             1
eating-disorders             1
sleep-improvement            1
addiction                    1
human-sexuality              1
diagnosis                    1
stress                       1
Name: count, dtype: int64
test data shape: 
                                                Context       topic  \
399  I've b

In [9]:
train_df = pd.read_csv("counsel_chat_train_balanced.csv")

# Filter the required columns
df = train_df[['questionText', 'answerText']]
# Rename the columns
df.columns = ['Context', 'Response']

df.head()

,Context,Response
0,I have so many issues to address. I have a his...,It is very common for people to have multiple ...
1,I have so many issues to address. I have a his...,"I've never heard of someone having ""too many i..."
2,I have so many issues to address. I have a his...,Absolutely not. I strongly recommending worki...
3,I have so many issues to address. I have a his...,Let me start by saying there are never too man...
4,I have so many issues to address. I have a his...,I just want to acknowledge you for the courage...


In [10]:

# Function to transform the row into desired format
def format_row(row):
    question = row['Context']
    answer = row['Response']
    formatted_string = f"<s>[INST] {question} [/INST] {answer}</s>"
    return formatted_string

# Apply the function to each row of the dataframe
df['Formatted'] = df.apply(format_row, axis=1)

# Rename the 'Formatted' column to 'Text'
new_df = df.rename(columns={'Formatted': 'Text'})

new_df



/tmp/ipykernel_34/220876148.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Formatted'] = df.apply(format_row, axis=1)


,Context,Response,Text
0,I have so many issues to address. I have a his...,It is very common for people to have multiple ...,<s>[INST] I have so many issues to address. I ...
1,I have so many issues to address. I have a his...,"I've never heard of someone having ""too many i...",<s>[INST] I have so many issues to address. I ...
2,I have so many issues to address. I have a his...,Absolutely not. I strongly recommending worki...,<s>[INST] I have so many issues to address. I ...
3,I have so many issues to address. I have a his...,Let me start by saying there are never too man...,<s>[INST] I have so many issues to address. I ...
4,I have so many issues to address. I have a his...,I just want to acknowledge you for the courage...,<s>[INST] I have so many issues to address. I ...
...,...,...,...
2594,I've been bullied for years and the teachers h...,To be bullied is very painful.I'm glad you kno...,<s>[INST] I've been bullied for years and the ...
2595,NaN,"Firstly, I am so sorry for the stressful thing...","<s>[INST] nan [/INST] Firstly, I am so sorry f..."
2596,I need to get on base to see my doctor. My ID ...,You should be able to get a replacement ID car...,<s>[INST] I need to get on base to see my doct...
2597,I need to get on base to see my doctor. My ID ...,Do you have any other form of identification? ...,<s>[INST] I need to get on base to see my doct...


In [11]:
## Split into Test and Train
from sklearn.model_selection import train_test_split
import pandas as pd

# Assuming 'final_df' is your DataFrame loaded with pd.read_csv("formatted_data.csv")
#train_df, test_df = train_test_split(new_df, test_size=0.2, random_state=42)  # Splitting 20% for testing

# Save the train and test datasets to CSV files
#test_df.to_csv('test_data.csv', index=False)
train_df = new_df.sample(frac=1, random_state=42)

test_df = pd.read_csv("counsel_chat_test_balanced.csv")

print(test_df.shape)
print(train_df.shape)

(176, 3)
(2599, 3)


In [12]:

# If you want to save the new dataframe to a CSV file:
train_df = train_df[['Text']]
train_df.to_csv('train_formatted_data.csv', index=False)
final_df = pd.read_csv("train_formatted_data.csv")
final_df.head(2)

,Text
0,<s>[INST] How do I decide if I should end my r...
1,<s>[INST] I struggle with depression as well a...


In [ ]:

training_dataset = load_dataset("csv", data_files="train_formatted_data.csv", split="train")
#print(dataset["Text"][400])
training_dataset

In [14]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
base_model = "microsoft/phi-2"
new_model = "phi-2-mental-health"

tokenizer = AutoTokenizer.from_pretrained(base_model, use_fast=True)
tokenizer.pad_token=tokenizer.eos_token
tokenizer.padding_side="right"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False,
)

model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    #use_flash_attention_2=True, # Phi does not support yet.
    trust_remote_code=True,
    flash_attn=True,
    flash_rotary=True,
    fused_dense=True,
    low_cpu_mem_usage=True,
    device_map={"": 0},
    revision="refs/pr/23",
)

model.config.use_cache = False
model.config.pretraining_tp = 1
#model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=True)
#model = prepare_model_for_kbit_training(model)

training_arguments = TrainingArguments(
    output_dir="Phi2",
    logging_dir='./results',
    num_train_epochs=2,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=32,
    evaluation_strategy="steps",
    eval_steps=2000,
    logging_steps=15,
    optim="paged_adamw_8bit",
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    save_steps=2000,
    warmup_ratio=0.05,
    weight_decay=0.01,
    max_steps=-1,
    load_best_model_at_end=True,
)



In [ ]:
peft_config = LoraConfig(
    r=8,
    lora_alpha=64,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules= ["Wqkv", "fc1", "fc2" ] # ["Wqkv", "out_proj", "fc1", "fc2" ], - 41M params
    # modules_to_save=["embed_tokens","lm_head"]
)

trainer = SFTTrainer(
    model=model,
    train_dataset=training_dataset,
    peft_config=peft_config,
    dataset_text_field="Text",
    max_seq_length=690,
    tokenizer=tokenizer,
    args=training_arguments,
)


In [17]:
gc.collect()
torch.cuda.empty_cache()

In [ ]:
trainer.train()

In [19]:
from peft import PeftModel, PeftConfig
from transformers import pipeline

'''
# Create the PEFT model
#model = PeftModel.from_pretrained(model, peft_config)
# Load the weights from the safetensor file 
state_dict = safetensors.torch.load_file('/kaggle/input/phi_weights/pytorch/phi_weights/1/model.safetensors')
#state_dict = safetensors.torch.load_file("/kaggle/input/phi/pytorch/phi2_weights/1/adapter_model.safetensors")
model.load_state_dict(state_dict, strict=False)
'''

#state_dict = safetensors.torch.load_file("path/to/your/trained/model.safetensor")
#model.load_state_dict(state_dict, strict=False)
model.eval()

tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2", use_fast=True)
tokenizer.pad_token=tokenizer.eos_token
tokenizer.padding_side="right"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=512)
generated_responses = []

n = 0

test_df = pd.read_csv("counsel_chat_test_balanced.csv")

# Fill NaN values in the 'Context' column with empty strings
test_df['Context'] = test_df['Context'].fillna('')

for index, row in tqdm(test_df.iterrows()):
    prompt = f"[INST] {row['Context'][:480]} [/INST]"
    reference = row['Response']
    
    result = pipe(prompt)
    generated_text = result[0]['generated_text']
    
    generated_responses.append({
            'Context': row['Context'],
            'topic': row['topic'],
            'Response': generated_text
        })

    if n < 5:
        print("Context:",row['Context'])
        print("generated_text:",generated_text)
        print("reference:",reference)
    
    #if n > 5:
    #    break
      
    n +=1
    
# Create a DataFrame from the generated responses
generated_df = pd.DataFrame(generated_responses)

# Write the DataFrame to an Excel file
generated_df.to_excel('Phi2_preds.xlsx', index=False)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


0it [00:00, ?it/s]

Context: I've been feeling worthless, unaccomplished, and so frustrated. I want to break things, hit myself, run away, kick, scream, and cry. I just want to be happy and outgoing again. I want to be able to take care of my daughter and live comfortably.
generated_text: [INST] I've been feeling worthless, unaccomplished, and so frustrated. I want to break things, hit myself, run away, kick, scream, and cry. I just want to be happy and outgoing again. I want to be able to take care of my daughter and live comfortably. [/INST] I'm sorry to hear that you are feeling this way.  It sounds like you are experiencing a lot of emotions and thoughts that are overwhelming.  I would encourage you to seek out a therapist who can help you work through these feelings and thoughts.  A therapist can help you identify the root causes of your feelings and thoughts and work with you to develop coping strategies to manage them.  I hope you find the help you need.</s>
   I'm sorry to hear that you are feelin

--- Logging error ---
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/logging/__init__.py", line 1100, in emit
    msg = self.format(record)
  File "/opt/conda/lib/python3.10/logging/__init__.py", line 943, in format
    return fmt.format(record)
  File "/opt/conda/lib/python3.10/logging/__init__.py", line 678, in format
    record.message = record.getMessage()
  File "/opt/conda/lib/python3.10/logging/__init__.py", line 368, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/opt/conda/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/opt/conda/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/opt/conda/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/opt/conda/lib/python3.10/site-packages/traitlets/config/application.py", line 104

In [ ]:

# Save the model weights
model.save_pretrained("ph2-finetuned1")
import torch

# For saving PyTorch model
torch.save(model.state_dict(), "ph2-finetuned.pth")

In [ ]:

try:
    trainer.model.push_to_hub(new_model, use_temp_dir=False)
except:
    print("An exception occurred")
    

## Evaluate Models

In [ ]:
pip install rouge 

In [ ]:
# Uninstall the previous installed nltk library
!pip install -U nltk

# This upgraded nltkto version 3.5 in which meteor_score is there.
!pip install nltk==3.5

In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('popular')


#ntk.download('omw-1.4')
nltk.download('wordnet2022')
#np = load('en_core_web_sm')
! cp -rf /usr/share/nltk_data/corpora/wordnet2022 /usr/share/nltk_data/corpora/wordnet 

In [ ]:
import pandas as pd
from nltk.translate.bleu_score import sentence_bleu
#from nltk.translate.meteor_score import single_meteor_score
from rouge import Rouge
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline , AutoModel
import torch
from scipy.spatial.distance import cosine
from nltk.tokenize import word_tokenize
from nltk.translate.bleu_score import SmoothingFunction
# Create an instance of the SmoothingFunction
chencherry = SmoothingFunction()

#  distinct_n_score and contextual_embedding_similarity functions are defined 
def contextual_embedding_similarity(text1, text2):
    """
    Computes the contextual embedding similarity between two texts.
    
    :param text1: The first text string.
    :param text2: The second text string.
    :param model_name: The pre-trained model to use.
    :return: The cosine similarity score.
    """
    # Load pre-trained model and tokenizer
    tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2", use_fast=True)
    tokenizer.pad_token=tokenizer.eos_token
    tokenizer.padding_side="right"
    model = AutoModel.from_pretrained("microsoft/phi-2")
    
    # Encode texts
    inputs1 = tokenizer(text1, return_tensors='pt', padding=True, truncation=True)
    inputs2 = tokenizer(text2, return_tensors='pt', padding=True, truncation=True)
    
    # Compute embeddings
    with torch.no_grad():
        embeddings1 = model(**inputs1).last_hidden_state.mean(dim=1)
        embeddings2 = model(**inputs2).last_hidden_state.mean(dim=1)
    
    # Compute cosine similarity
    cosine_similarity = 1 - cosine(embeddings1[0].numpy(), embeddings2[0].numpy())
    
    return cosine_similarity


def distinct_n_score(text, n=1):
    """
    Computes the Distinct-n score for the given text.
    
    :param text: The input text string.
    :param n: The n-gram length.
    :return: The Distinct-n score as a float.
    """
    if not text or n <= 0:
        return 0
    
    # Tokenize the text into words
    words = text.split()
    
    # Generate n-grams
    n_grams = set([' '.join(words[i:i+n]) for i in range(len(words)-n+1)])
    
    # Calculate Distinct-n score
    distinct_n_score = len(n_grams) / float(len(words) - n + 1)
    
    return distinct_n_score


# Prepare tokenizer and model for similarity computation (Choose an appropriate model)
embeddeding_model_name = 'bert-base-uncased'
#tokenizer = AutoTokenizer.from_pretrained(embeddeding_model_name)
embeddingmodel = AutoModelForCausalLM.from_pretrained(embeddeding_model_name)


from nltk.translate import meteor_score


# Initialize ROUGE scorer
rouge_scorer = Rouge()

# Placeholder for scores
scores_dict = {
    'BLEU': [],
    'ROUGE-L': [],
    'METEOR': [],
    'Distinct-1': [],
    'Distinct-2': [],
    'Contextual Embedding Similarity': []
}
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=512)

n = 0
for index, row in tqdm(test_df.iterrows()):
    prompt = f"[INST] {row['Context']} [/INST]"
    reference = row['Response']
    result = pipe(prompt)
    generated_text = result[0]['generated_text']
    
    if n < 10:
        print("Context:",row['Context'])
        print("generated_text:",generated_text)
        print("reference:",reference)
      
    # Compute BLEU
    # Computing the BLEU score, apply the smoothing function
    bleu_score = sentence_bleu([reference.split()], generated_text.split(), smoothing_function=chencherry.method1)
    scores_dict['BLEU'].append(bleu_score)
    
    # Compute ROUGE
    rouge_scores = rouge_scorer.get_scores(generated_text, reference)[0]['rouge-l']['f']
    scores_dict['ROUGE-L'].append(rouge_scores)
    
    # Tokenize the reference and generated text for METEOR score computation
    #tokenized_reference = nltk.tokenize.word_tokenize(reference)
    #tokenized_generated_text = nltk.tokenize.word_tokenize(generated_text)

    # Compute METEOR with tokenized inputs
    #meteor_score_value = meteor_score.meteor_score([' '.join(tokenized_reference)], ' '.join(tokenized_generated_text))
    meteor_score_value = meteor_score.meteor_score([reference], generated_text)

    #meteor_score_value = meteor_score.meteor_score([tokenized_reference], tokenized_generated_text)
    scores_dict['METEOR'].append(meteor_score_value)
    

    # Compute Distinct-n Scores
    distinct_1 = distinct_n_score(generated_text, 1)
    distinct_2 = distinct_n_score(generated_text, 2)
    scores_dict['Distinct-1'].append(distinct_1)
    scores_dict['Distinct-2'].append(distinct_2)
    
    # Compute Contextual Embedding Similarity
    similarity_score = contextual_embedding_similarity(reference, generated_text, model_name=embeddeding_model_name)
    scores_dict['Contextual Embedding Similarity'].append(similarity_score)
    n +=1
    
# Calculate average scores
average_scores = {metric: sum(scores) / len(scores) for metric, scores in scores_dict.items()}

# Print average scores
for metric, average_score in average_scores.items():
    print(f"Average {metric}: {average_score}")
    

In [ ]:
gc.collect()
torch.cuda.empty_cache()

In [21]:
import os
from IPython.display import display, HTML  # Importing HTML module

# Get the current working directory
current_directory = os.getcwd()

# Specify the path to the Excel file
excel_file_path = os.path.join(current_directory, 'Phi2_preds.xlsx')
print(excel_file_path)
# Check if the file exists
if os.path.exists(excel_file_path):
    print("Excel file found. Initiating download...")
    
    # Create a download link
    download_link = '<a href="{}" download>Click here to download Phi2_preds.xlsx</a>'.format(excel_file_path)
    
    # Display the download link
    display(HTML(download_link))
    
    print("Download complete.")
else:
    print("Phi2_preds.xlsx file not found in the current directory.")


/kaggle/working/Phi2_preds.xlsx
Excel file found. Initiating download...


Download complete.


In [ ]:
best_ckpt_path = trainer.state.best_model_checkpoint
print(best_ckpt_path)